In [9]:
from data_retrieval import alpaca
from data_retrieval import crypto_market
from data_retrieval import getbinance
from functions_graphs import functions
from user_input import risk_assesment
import pandas as pd

In [10]:
# user_input = ra.get_user_risk_tolerance()

In [45]:
def get_crypto_dict():
    # Returns a dictionary with ticker as index and its dataframe as a value
    cryptos = ['ADA', 'BNB', 'BTC', 'DOT', 'ETH', 'LINK', 'LTC', 'VET', 'XLM', 'XRP']
    crypto_dict = {}
    for ticker in cryptos:
        df = getbinance.Binance(ticker, 3)
        df = pd.DataFrame(df)
        crypto_dict[ticker] = df

    return crypto_dict

In [46]:
dictionary = get_crypto_dict()

In [47]:
def get_std(crypto_dfs):
    # Returns a dictionary of all crypto stds
    cryptos = ['ADA', 'BNB', 'BTC', 'DOT', 'ETH', 'LINK', 'LTC', 'VET', 'XLM', 'XRP']
    crypto_std = {}
    for ticker in cryptos:
        df = crypto_dfs[ticker]
        c_name = f"{ticker}USDT_Close"
        funct = functions.Functions(df, c_name)
        df = funct.daily_returns()
        df.dropna(inplace=True)

        crypto_std[ticker] = funct.standard_deviation()
#         display(df.head())
    return crypto_std

In [48]:
crypto_std = get_std(dictionary)

In [15]:
# Make a dictionary, save the main key as the ticker, in the values save the std and the percent change series in nexted dictionaries

In [49]:
# 0.0 -.057 Low
# .057 - .073 Med
# .073 - 1 High
def sort_crypto_std(crypto_std):
    # Returns a dictionary with all crypto std's sorted from low to high risk
    high_risk = {}
    med_risk = {}
    low_risk = {}
    for ticker, std in crypto_std.items():
        if float(std) <= 0.057:
            low_risk[ticker] = std
        if float(std) > 0.057 and float(std) <= 0.073:
            med_risk[ticker] = std
        if float(std) > 0.073:
            high_risk[ticker] = std


    return {"low":low_risk, "med":med_risk, "high":high_risk}

In [50]:
sorted_std = sort_crypto_std(crypto_std)
for t,v in sorted_std['low'].items():
    print(f"Low Risk Cryptos - {t}:{v}")
print("----------")
for t,v in sorted_std['med'].items():
    print(f"Medium Risk Cryptos - {t}:{v}")  
print("----------")
for t,v in sorted_std['high'].items():
    print(f"High Risk Cryptos - {t}:{v}")

Low Risk Cryptos - BTC:0.039394843988365534
Low Risk Cryptos - ETH:0.052361023385229806
Low Risk Cryptos - LTC:0.055038082950316425
----------
Medium Risk Cryptos - ADA:0.05999963720001747
Medium Risk Cryptos - BNB:0.05977418155407327
Medium Risk Cryptos - LINK:0.07157632906865609
Medium Risk Cryptos - VET:0.06852191988742666
Medium Risk Cryptos - XLM:0.06330666486672644
Medium Risk Cryptos - XRP:0.06490930907480714
----------
High Risk Cryptos - DOT:0.08365198580869644


{'low': {'BTC': 0.03945304771366324,
  'ETH': 0.05244884363519114,
  'LTC': 0.05512365630029008},
 'med': {'ADA': 0.060055934547537264,
  'BNB': 0.059868182427903385,
  'LINK': 0.07171179932422474,
  'VET': 0.06857860252382497,
  'XLM': 0.063401144014262,
  'XRP': 0.06500505286508543},
 'high': {'DOT': 0.07859867388294556}}